In [72]:
import pandas as pd
import numpy as np

In [73]:
def repl(string):
    string1 = string.replace(']','').replace('[', '').replace('\'','')
    return string1

In [106]:
def one_decimal(rating):
    return round(rating, 1)

def categorize_ratings(df, col):
    df[col] = df[col].apply(one_decimal)
    return df
def lower(text):
    return text.lower()

def lower_cols(df, cols):
    for col in cols:
        df[col] = df[col].apply(lower)
    return df

def remove_spaces(text):
    text = text.replace(' ', '', 10)
    return text

def join_names(df, col):
    df[col] = df[col].apply(remove_spaces)
    return df

def read_books_info(filespath='../data/small_dataset/'):
    pattern = 'revs_*.csv'
    print('Pattern :', pattern )

    files = os.listdir(filespath) 
    dfs_files = []
    for name in files: 
        if fnmatch.fnmatch(name, pattern):
            dfs_files.append(name)
    dfs_files
    dfs_paths = []
    for file in dfs_files:
        dfs_paths.append(filespath + file)

    revs_lst = []
    for file in dfs_paths:
        try:
            revs_lst.append(pd.read_csv(file, usecols=[1,2,3]))
        except:
            print(f'file {file} failed')
    print(len(revs_lst))
    revs_df = pd.concat(revs_lst,axis=0)
    
   
    return df

In [108]:
books_df = pd.read_csv('../data/big_data_temp/gr_books_df_0.csv')
books_df['author_names'] = books_df['author_names'].apply(repl)
books_df = categorize_ratings(books_df,'avg_rating')
books_df = books_df[['isbn', 'book_title', 'avg_rating', 'author_names', 'book_description']]
books_df = lower_cols(books_df, ['book_title', 'author_names', 'book_description'])
books_df = join_names(books_df, 'author_names')

In [112]:
books_df['content'] = books_df['book_title'] + ' ' + books_df['author_names'] + ' ' + books_df['book_description']

In [113]:
books_df['content']

0     a brief history of time stephenhawking in the ...
1     a short history of nearly everything billbryso...
2     the selfish gene richarddawkins the selfish ge...
3     cosmos carlsagan cosmos has 13 heavily illustr...
4     the man who mistook his wife for a hat and oth...
                            ...                        
80    superintelligence: paths, dangers, strategies ...
81    the character of physical law richardp.feynman...
82    other minds: the octopus, the sea, and the dee...
83    the tell-tale brain: a neuroscientist's quest ...
84    the big picture: on the origins of life, meani...
Name: content, Length: 85, dtype: object

In [146]:
books_df.head(3)

,isbn,book_title,avg_rating,author_names,book_description,content,ability,accessible,acclaimed,advances,...,view,war,way,wit,work,works,world,years,york,young
0,0553380168,a brief history of time,4.2,stephenhawking,in the ten years since its publication in 1988...,a brief history of time stephenhawking in the ...,0.0,0.0,0.0,0.15334,...,0.0,0.0,0.000000,0.0,0.105324,0.0,0.000000,0.298464,0.0,0.0
1,076790818X,a short history of nearly everything,4.2,billbryson,"in bryson's biggest book, he confronts his gre...",a short history of nearly everything billbryso...,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.092611,0.000000,0.0,0.0
2,0199291152,the selfish gene,4.1,richarddawkins,the selfish gene: 30th anniversary edition—wit...,the selfish gene richarddawkins the selfish ge...,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.338669,0.0,0.110650,0.0,0.070817,0.104518,0.0,0.0


In [138]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 1), min_df=0, stop_words='english', max_features=200)
tfidf_matrix = tf.fit_transform(books_df['content'])

# tf.stop_words_
# tf.vocabulary_
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tf.get_feature_names())
books_df = pd.concat([books_df,tfidf_df], axis=1)
features = tf.get_feature_names()

In [139]:
reviews_df = pd.read_csv('../data/small_dataset/reviews.csv')

In [158]:
num_users = len(reviews_df.UserID.unique())
num_books = len(reviews_df.ISBN.unique())
print(num_users, num_books, num_users*num_books)
isbns = list(reviews_df.ISBN)
users = list(reviews_df.UserID)

user_isbn = zip(users, isbns)

4038 93 375534


In [142]:
from lightfm import LightFM
from lightfm.data import Dataset

In [145]:
dataset = Dataset()
dataset.fit(reviews_df['UserID'],reviews_df['ISBN'])

In [148]:
dataset.fit_partial(items=books_df['isbn'], users=reviews_df['UserID'], item_features=features, user_features=None)

In [160]:
(interactions, weights) = dataset.build_interactions(user_isbn)

In [147]:
item_features = dataset.build_item_features(items = books_df['isbn'], item_features = features)

TypeError: build_item_features() got an unexpected keyword argument 'items'

In [ ]:
X = zip(reviews_df['ISBN']), reviews_df['UserID']

In [46]:
avg_ratings = list(books_df['avg_rating'].unique())
book_titles = list(books_df['book_title'].unique())
author_names = list(books_df['book_title'].unique())
cat_features = avg_ratings + book_titles + author_names